In [10]:
!pip install bbknn

ModuleNotFoundError: No module named 'bbknn'

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from collections import Counter
import os
import anndata as ann
import bbknn

  Using cached bbknn-1.6.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached Cython-3.0.10-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached annoy-1.17.3.tar.gz (647 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached bbknn-1.6.0-py3-none-any.whl (14 kB)
Using cached Cython-3.0.10-cp311-cp311-win_amd64.whl (2.8 MB)
  Running setup.py clean for annoy
Failed to build annoy


'DOSKEY' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      E:\software\anaconda3\Lib\site-packages\setuptools\__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\annoy
      copying annoy\__init__.py -> build\